# 1. Peridically fetch traffic trends and attack spikes from clodfare and/or IPS from abuse IPDP
# 2. Classify IPS with high DDos confidence score using machine learning
# 3, Remote IPs to coordinate and show them in fromtend globe

In [69]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [3]:
API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

PARAMS = {
    "aggInterval": "1h",
    "dateRange": "1d",
    "format": "json"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {})
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Layer 3 URL
vector_url = "https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/vector"
ip_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/ip_version"
protocol_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/protocol"
bitrate_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/bitrate"
duration_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/duration"

vector_data = fetch_data(vector_url)
ip_data=fetch_data(ip_url)
protocol_data=fetch_data(protocol_url)
bitrate_data=fetch_data(bitrate_url)
duration_data=fetch_data(duration_url)


df_vector = pd.DataFrame(vector_data)
df_ip = pd.DataFrame(ip_data)
df_protocol = pd.DataFrame(protocol_data)
df_bitrate = pd.DataFrame(bitrate_data)
df_duration = pd.DataFrame(duration_data)


In [26]:
to_melt=[vector_data,ip_data,protocol_data,bitrate_data,duration_data]

In [37]:
data=vector_data["serie_0"]
df=pd.DataFrame(data)
df_vector=df.melt(
    id_vars=["timestamps"],
    var_name="attack_type",
    value_name="value"
)

In [38]:
data=ip_data["serie_0"]
df=pd.DataFrame(data)
df_ip=df.melt(
    id_vars=["timestamps"],
    var_name="version"
)

In [53]:
data=protocol_data["serie_0"]
df=pd.DataFrame(data)
df_protocol=df.melt(
    id_vars=["timestamps"],
    var_name="protocol"
)

In [52]:
data=bitrate_data["serie_0"]
df=pd.DataFrame(data)
df_bitrate=df.melt(
    id_vars=["timestamps"],
    var_name="bitrate"
)


In [48]:
data=duration_data["serie_0"]
df=pd.DataFrame(data)
df_duration=df.melt(
    id_vars=["timestamps"],
    var_name="duration"
)


In [79]:
df_duration
df_bitrate
df_protocol
# df_ip
# df_vector

,timestamps,protocol,value
0,2025-08-15 03:00:00+00:00,UDP,70.20109
1,2025-08-15 04:00:00+00:00,UDP,77.16712
2,2025-08-15 05:00:00+00:00,UDP,75.399461
3,2025-08-15 06:00:00+00:00,UDP,79.927863
4,2025-08-15 07:00:00+00:00,UDP,81.449619
...,...,...,...
91,2025-08-15 22:00:00+00:00,GRE,7.844326
92,2025-08-15 23:00:00+00:00,GRE,0.0
93,2025-08-16 00:00:00+00:00,GRE,0.0
94,2025-08-16 01:00:00+00:00,GRE,0.0


In [78]:
for df in [df_duration, df_bitrate, df_protocol, df_ip, df_vector]:
    df['timestamps'] = pd.to_datetime(df['timestamps'], errors='coerce')

In [ ]:
df_protocol